In [ ]:
# https://huggingface.co/stabilityai/stablelm-2-1_6b

import time
import torch
# https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM
# https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# Instantiates Arcade100kTokenizer (not yet documented as of 2024-01-20)
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-2-1_6b", trust_remote_code=True)
# print("Using %s" % tokenizer)

In [ ]:
# Instantiates StableLMEpochForCausalLM
if torch.backends.mps.is_available():
    model = AutoModelForCausalLM.from_pretrained(
      "stabilityai/stablelm-2-1_6b", trust_remote_code=True)
    model = model.to(torch.device("mps"))
else:
    model = AutoModelForCausalLM.from_pretrained(
      "stabilityai/stablelm-2-1_6b",
      trust_remote_code=True,
      torch_dtype="auto",
    )
    if torch.backends.cuda.is_available():
        model.cuda()
# print("Using %s" % model)

In [ ]:
input = "Write a poem about the sky"

start = time.time()
tokens = model.generate(
  **tokenizer(input, return_tensors="pt").to(model.device),
  max_new_tokens=128,
  temperature=0.70,
  top_p=0.95,
  do_sample=True,
  streamer=TextStreamer(tokenizer, skip_prompt=True),
  pad_token_id=tokenizer.eos_token_id,
)
decoded = tokenizer.decode(tokens[0], skip_special_tokens=True)
words = len(decoded.split())
duration = time.time()-start
print(f"Generated {words} words in {duration:.1f}s, {(words/duration):.1f} words/s")